In [0]:
#Connection with Google Drive

#from google.colab import drive
#drive.mount('/content/gdrive')
#!cp -rv /content/train_processed.zip /content/gdrive/My\ Drive/train_processed.zip
#!rm /content/gdrive/My\ Drive/kaggle_data/train_processed.zip
#!cp -rv /content/gdrive/My\ Drive/train_processed.zip /content/gdrive/My\ Drive/kaggle_data/train_processed.zip

In [0]:
# Get data from Kaggle
!pip install kaggle
!mkdir ~/.kaggle
!mkdir /content/.kaggle
!ls ~/.kaggle

import json
token = {"username":"mielgosez", "key":"46ef90eb86d527d54aca090759ccbef9"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
    
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!chmod 600 /root/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

print("1. Downloading datasets")
!kaggle competitions download -c histopathologic-cancer-detection -p /content/

print("2.  Moving train set to train folder")
!mkdir /content/train
!unzip -qq /content/train.zip -d /content/train
print("3. Moving test set to test folder")
!mkdir /content/test
!unzip -qq /content/test.zip -d /content/test


print("4. Moving files to their corresponding folders")
!mkdir /content/train/0
!mkdir /content/train/1
import pandas as pd
import shutil
!unzip -qq /content/train_labels.csv.zip
file_assign = pd.read_csv("/content/train_labels.csv") 
file_assign = file_assign.values
for item in file_assign:
  if item[1] == 0:
    shutil.move("/content/train/"+item[0]+".tif", 
                "/content/train/0/"+item[0]+".tif")
  else:
    shutil.move("/content/train/"+item[0]+".tif", 
                "/content/train/1/"+item[0]+".tif")

- path is now set to: {/content}
1. Downloading datasets
  0% 0.00/1.33M [00:00<?, ?B/s]
100% 1.33M/1.33M [00:00<00:00, 95.4MB/s]
  0% 0.00/5.10M [00:00<?, ?B/s]
100% 5.10M/5.10M [00:00<00:00, 83.5MB/s]
 99% 1.29G/1.30G [00:09<00:00, 206MB/s]
100% 1.30G/1.30G [00:09<00:00, 146MB/s]
100% 4.98G/4.98G [00:38<00:00, 138MB/s]
100% 4.98G/4.98G [00:38<00:00, 140MB/s]
2.  Moving test set to train folder
3. Moving test set to test folder
4. Moving files to their corresponding folders


In [0]:
# Verify that the 
import os
path, dirs, files = next(os.walk("/content/train/0"))
file_count = len(files)
print("Number of files in train folder cat 0:"+ str(file_count))

path, dirs, files = next(os.walk("/content/train/1"))
file_count = len(files)
print("Number of files in train folder cat 1:"+ str(file_count))

Number of files in train folder cat 0:130908
Number of files in train folder cat 1:89117


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/train"
test_dir = "/content/test"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=5,
        horizontal_flip=True,
        validation_split=0.2)

val_datagen = ImageDataGenerator(
        rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),
        color_mode="rgb",
        batch_size=64,
        class_mode='categorical',
        subset='training')

validation_generator = val_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),
        color_mode="rgb",
        batch_size=64,
        class_mode='categorical',
        subset='validation')

Using TensorFlow backend.


Found 176021 images belonging to 2 classes.
Found 44004 images belonging to 2 classes.


In [0]:
from PIL import Image
import pandas as pd

file_assign = pd.read_csv("/content/train_labels.csv") 
file_assign = file_assign.values

image_sizes = []
for item in file_assign:
  if item[1] == 0:
    im = Image.open("/content/train/0/"+item[0]+".tif")
  else:
    im = Image.open("/content/train/1/"+item[0]+".tif")
  image_sizes.append(im.size)

In [0]:
ps = pd.Series([tuple(i) for i in image_sizes])
counts = ps.value_counts()
print(counts)

(96, 96)    220025
dtype: int64


In [0]:
from keras import models
from keras import layers
from keras import regularizers
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu', 
                       kernel_regularizer=regularizers.l2(0.01),
                       activity_regularizer=regularizers.l1(0.01)))
model.add(layers.Dropout(0.05))
model.add(layers.Dense(32, activation='relu', 
                       kernel_regularizer=regularizers.l2(0.01),
                       activity_regularizer=regularizers.l1(0.01)))
model.add(layers.Dense(2, activation='sigmoid',
                       kernel_regularizer=regularizers.l2(0.01),
                       activity_regularizer=regularizers.l1(0.01)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                294976    
__________

In [0]:
n_epochs = 20
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, 
                               verbose=0, mode='auto', baseline=None,
                               restore_best_weights=False)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print(validation_generator.samples // n_epochs)
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // n_epochs,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // n_epochs,
    epochs = n_epochs*3,
    callbacks = [reduce_lr, early_stopping])

2200
Instructions for updating:
Use tf.cast instead.
Epoch 1/60
8801/8801 [==============================] - 3641s 414ms/step - loss: 0.8135 - acc: 0.5944 - val_loss: 0.6952 - val_acc: 0.5949
Epoch 2/60
8801/8801 [==============================] - 3505s 398ms/step - loss: 0.6882 - acc: 0.5950 - val_loss: 0.6841 - val_acc: 0.5951
Epoch 3/60
8801/8801 [==============================] - 3375s 383ms/step - loss: 0.6825 - acc: 0.5947 - val_loss: 0.6815 - val_acc: 0.5943
Epoch 4/60
6415/8801 [====================>.........] - ETA: 13:51 - loss: 0.6804 - acc: 0.5952

KeyboardInterrupt: ignored

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
